## Generating task infos

To plot Figure 3 (the task overview) we need to transform task features.

In [1]:
import mml.interactive
mml.interactive.init()
from mml.api import Keyword
from mml_tf.tasks import all_tasks, train_tasks, test_tasks, task_infos
from mml_tf.distances import transformed_task_infos
from mml_tf.paths import FIG_PATH
import pandas as pd

/home/scholzpa/Documents/development/gitlab/mml/src/mml/interactive/__init__.py:49: UserWarning: Did not provide a "env_path", neither found set "MML_ENV_PATH" variable, you might need to provide "env_path" to "init" in order to use "mml" interactively in a jupyter/ipython setting.
  warnings.warn(
/home/scholzpa/miniconda3/envs/mml/lib/python3.8/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


 _____ ______   _____ ______   ___
|\   _ \  _   \|\   _ \  _   \|\  \
\ \  \\\__\ \  \ \  \\\__\ \  \ \  \
 \ \  \\|__| \  \ \  \\|__| \  \ \  \
  \ \  \    \ \  \ \  \    \ \  \ \  \____
   \ \__\    \ \__\ \__\    \ \__\ \_______\
    \|__|     \|__|\|__|     \|__|\|_______|
         ____  _  _    __  _  _  ____  _  _
        (  _ \( \/ )  (  )( \/ )/ ___)( \/ )
         ) _ ( )  /    )( / \/ \\___ \ )  /
        (____/(__/    (__)\_)(_/(____/(__/
Interactive MML API initialized.


First we generate some descriptive statistics for the task attributes.

In [11]:
# statistics
def get_percentiles(attr: str) -> str:
    assert hasattr(task_infos, attr)
    vals = pd.Series(list(getattr(task_infos, attr).values()))
    desc = vals.describe()
    return f'{attr=}: min={desc.loc["min"]}, q1={desc.loc["25%"]}, q2={desc.loc["50%"]}, q3={desc.loc["75%"]}, max={desc.loc["max"]}, mean={desc.loc["mean"]}, std={vals.std()}'

In [13]:
for attr in ['num_classes', 'num_samples', 'dimensions', 'imbalance_ratios']:
    print(get_percentiles(attr))

attr='num_classes': min=2.0, q1=2.0, q2=2.0, q3=5.0, max=257.0, mean=11.985915492957746, std=36.558560833563014
attr='num_samples': min=170.0, q1=1572.5, q2=5718.0, q3=40673.0, max=122138.0, mean=28163.521126760563, std=36595.276309873334
attr='dimensions': min=4.0, q1=9.0, q2=13.0, q3=14.0, max=25.0, mean=12.04225352112676, std=4.254196649757608
attr='imbalance_ratios': min=1.0, q1=1.3456414066188114, q2=2.878428465530022, q3=14.277188328912466, max=171.33333333333334, mean=14.838794320313594, std=29.773240229472243


These numbers have been reported in the legend and caption of Figure 3.

In [6]:
task_overview_details = []
structs = mml.interactive.get_task_structs(all_tasks)

tag_groups = {
                tag.value: [tag]
                for tag in [
                    Keyword.DERMATOSCOPY,
                    Keyword.LARYNGOSCOPY,
                    Keyword.GASTROSCOPY_COLONOSCOPY,
                    Keyword.LAPAROSCOPY,
                    Keyword.NATURAL_OBJECTS,
                    Keyword.HANDWRITINGS,
                    Keyword.CATARACT_SURGERY,
                    Keyword.FUNDUS_PHOTOGRAPHY,
                    Keyword.MRI_SCAN,
                    Keyword.X_RAY,
                    Keyword.CT_SCAN,
                    Keyword.CLE,
                    Keyword.ULTRASOUND,
                    Keyword.CAPSULE_ENDOSCOPY,
                ]
            }
# convert tag groups to task clusters
task_clusters = {
    group_name: [task.name for task in structs if any([tag in task.keywords for tag in group_tags])]
    for group_name, group_tags in tag_groups.items()
}
clusters = sorted(list(task_clusters.keys()))
legend_map = {ix: cluster_name for ix, cluster_name in enumerate(clusters)}
task_mapping = {name: cluster for cluster in task_clusters.keys() for name in task_clusters[cluster]}
color_map = {task.name: clusters.index(task_mapping[task.name]) for task in structs}
# color_map, legend_map = plot_2D.create_color_mapping(task_list=structs, criteria='domain', task_clusters=None)
REPLACEMENTS = {'cataract_surgery': 'ophthalmic microscopy',
                'ct_scan': 'CT', 'fundus_photography': 'fundus photography',
                'gastroscopy_colonoscopy': 'gastro & colonoscopy',
                'mri_scan': 'MRI', 'natural_objects': 'natural images', 'x_ray': 'X-ray',
                }
for k in legend_map:
    if legend_map[k] in REPLACEMENTS:
        legend_map[k] = REPLACEMENTS[legend_map[k]]
domains = {t: legend_map[color_map[t]] for t in all_tasks}
for t in train_tasks + test_tasks:
    _printable = t.replace('_', ' ').replace('lapgyn4', 'LapGyn4').replace('sklin2 skin lesions', 'SKLIN2').replace(
        'object classification', '').replace('svhn', 'SVHN').replace('cholec80', 'Cholec80').replace('hyperkvasir',
                                                                                                     'HyperKvasir').replace(
        'mura xr', 'MURA').replace('identify nbi infframes', 'NBI-InfFrames').replace('laryngeal tissues',
                                                                                      'Laryngeal cancerous tissue').replace(
        'stanford dogs image categorization', 'Stanford dogs').replace('digit classification', '').replace('emnist',
                                                                                                           'EMNIST').replace(
        'mnist', 'MNIST').replace('caltech', 'Caltech').replace('cifar', 'CIFAR').replace('skin lesions', '').replace(
        'idle action recognition', 'CatRelComp').replace('chexpert', 'CheXpert').replace('barretts esophagus diagnosis',
                                                                                         'AIDA-E Barrett\'s esophagus').replace(
        'brain tumor classification', 'kaggle Brain Tumor dataset').replace('nerthus bowel cleansing quality',
                                                                            'Nerthus').replace(
        'mednode melanoma classification', 'MED-NODE').replace('crawled covid ct classification',
                                                               'COVID-CT-Dataset').replace(
        'ph2-melanocytic-lesions-classification', 'PH2').replace('covid xray classification',
                                                                 'kaggle COVID X-Ray dataset').replace(
        'isic20 melanoma classification', 'ISIC20').replace('deep drid', 'DeepDRiD').replace('kvasir capsule',
                                                                                             'Kvasir-Capsule').replace(
        'bean plant disease classification', 'ibean').replace('aptos19 blindness detection',
                                                              'APTOS 2019 Blindness Detection').replace(
        'eye condition classification', 'kaggle cataract dataset').replace('breast cancer classification v2',
                                                                           'Dataset of breast ultrasound images').replace(
        'shenzen chest xray tuberculosis', 'Shenzhen Hospital CXR Set').replace('pneumonia classification',
                                                                                'Zhang Chest X-Ray Images')
    group = None
    if 'cholec80' in t:
        group = 'Cholec80'
    elif 'deep_drid' in t:
        group = 'deepdrid'
    elif 'chexpert' in t:
        group = 'chexpert'
    _info = {'name': t, 'train': t in train_tasks, 'printable': _printable.strip(),
             'samples': transformed_task_infos.num_samples[t],
             'classes': transformed_task_infos.num_classes[t], 'dimension': transformed_task_infos.dimensions[t],
             'imbalance': transformed_task_infos.imbalance_ratios[t], 'domain': domains[t], 'group': group}
    task_overview_details.append(_info)
pd.DataFrame(task_overview_details).to_csv(FIG_PATH / 'advanced_task_infos.csv')

These transformed properties have been used to generate the bar charts in Figure 3.